In [3]:
import cv2
import numpy as np
import mss
import ctypes
from ctypes import wintypes
    
INPUT_MOUSE = 0
MOUSEEVENTF_MOVE = 0x0001
MOUSEEVENTF_LEFTDOWN = 0x0002
MOUSEEVENTF_LEFTUP = 0x0004
    #structure de l'api
class Input(ctypes.Structure):
    _fields_ = [("type", wintypes.DWORD),
                ("mi", ctypes.c_ulong)]
class MouseInput(ctypes.Structure):
    _fields_ = [("dx", wintypes.LONG),
                ("dy", wintypes.LONG),
                ("mouseData", wintypes.DWORD),
                ("dwFlags", wintypes.DWORD),
                ("time", wintypes.DWORD),
                ("dwExtraInfo", wintypes.ULONG)]
    #utilisation de l'api windobe pour faire une fausse entrer
def send_input(inputs):
    nInputs = len(inputs)
    size_input = ctypes.sizeof(Input)
    arr_inputs = Input * nInputs
    pInputs = arr_inputs(*inputs)
    cb_size = ctypes.c_int(ctypes.sizeof(pInputs))
    ctypes.windll.user32.SendInput(nInputs, ctypes.byref(pInputs), cb_size)
    #deplacement de sourie avec x et y
def move_mouse(dx, dy):
    inputs = []
    mi = MouseInput(dx, dy, 0, MOUSEEVENTF_MOVE, 0, ctypes.cast(None, wintypes.LPVOID))
    inputs.append(Input(INPUT_MOUSE, mi))
    send_input(inputs)
    click_left()
    #faux click gauche
def click_left():
    inputs = []
    mi_down = MouseInput(0, 0, 0, MOUSEEVENTF_LEFTDOWN, 0, None)
    mi_up = MouseInput(0, 0, 0, MOUSEEVENTF_LEFTUP, 0, None)
    inputs.append(Input(INPUT_MOUSE, mi_down))
    inputs.append(Input(INPUT_MOUSE, mi_up))
    send_input(inputs)
    #pour mon yolo<3
net = cv2.dnn.readnet("yolov4-tiny-custom_best.weights", "yolov4-tiny-custom.cfg")
with mss.mss() as sct:
    while True:
        img = cv2.cvtcolor(np.array(sct.shot()), cv2.COLOR_BGR2RGB)
        bob = cv2.dnn.bobfromimage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(bob)
        outs = net.forward(net.getunconnectedoutlayersnames())
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * img.shape[1])
                    center_y = int(detection[1] * img.shape[0])
                    w = int(detection[2] * img.shape[1])
                    h = int(detection[3] * img.shape[0])
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    move_mouse(x, y)


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src data type = 19 is not supported
>  - Expected Ptr<cv::UMat> for argument 'src'
